Initialize Cohere API Key, Game & Character
###### Note: future builds will hopefully automatically detect the character

In [1]:
from langchain_cohere import ChatCohere
import getpass
import os
import json
with open(f'api.txt', errors='ignore') as f:
    api_key = f.read()
model = ChatCohere(cohere_api_key=api_key)

game = "elden_ring"
character = "Varre"
with open(f"{game}/characters/{character}/id.txt", errors='ignore') as f:
    conversation_id = f.read()
config = {"configurable": {"thread_id": conversation_id}}

c:\Users\ryanz\Desktop\LLM-Based-NPC\.venv\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


Initialize RAG for Long Term Conversational Memory
###### Note:

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_chroma import Chroma
from langchain_core.embeddings import Embeddings
from uuid import uuid4
import chromadb

embeddings = CohereEmbeddings(cohere_api_key=api_key, model="embed-english-v3.0", user_agent='langchain')
vector_store = Chroma(
    collection_name=f"{character}_conversation_history",
    embedding_function=embeddings,
    persist_directory=f"{game}/characters/{character}/conversation_vectordbs",
)
retriever = vector_store.as_retriever(
    search_kwargs={'k': 10}
)

In [3]:
rag_query = "Hello"
print(f"Rag Query: {rag_query}")
documents = retriever.invoke(rag_query)

for res in documents:
    print(f"{res.page_content}")

Rag Query: Hello


Connect conversation state to an external directory
###### Note: If the directory does not exist it will create one

In [4]:
import sqlite3
conn = sqlite3.connect(":memory:")

db_path = f"{game}/characters/{character}/state_db_with_rag/history.db"
conn = sqlite3.connect(db_path, check_same_thread=False)

from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver(conn)

Initialize LLM Graph Workflow

In [10]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END

from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, AIMessage, trim_messages, RemoveMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph.message import add_messages

from typing import Sequence
from typing_extensions import Annotated, TypedDict

from typing import List
from typing_extensions import TypedDict

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are {character} from {game}.
            {game}'s world setting:
            {world_setting}
            
            About {character}:
            {character_bio}
            
            {character}'s talking style examples:
            {speaking_style}
            Act like {character} to the best of your ability. Do not hallucinate.""",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

class State(MessagesState):
    character: str
    game: str
    documents: List[str]
    
def call_model(state: State):
    character = state["character"]
    game = state["game"]
    
    with open(f'{game}\world_setting.txt', errors='ignore') as f:
        world_setting = f.read()
    
    with open(f'{game}\characters\{character}\character_bio.txt', errors='ignore') as f:
        character_bio = f.read()
    
    with open(f'{game}\characters\{character}\speaking_style.txt', errors='ignore') as f:
        speaking_style = f.read()
            
    chain = prompt | model
    
    print(f"\nDisplaying message type order:")
    for message in state["messages"]:
        if isinstance(message, HumanMessage):
            print(f"HumanMessage")
        elif isinstance(message, AIMessage):
            print(f"AIMessage")
    print(f"\n")
    
    documents = state.get("documents", [])
    if documents:
        print(f"Documents found!")
        system_message = f"Previous conversations that may aid your response:\n{documents}"
        messages = [SystemMessage(content=system_message)] + state["messages"]        
    else:
        messages = state["messages"]
    
    response = chain.invoke(
        {"messages": messages, "character": character, "game": game, "world_setting": world_setting, "character_bio": character_bio, "speaking_style": speaking_style}
    )
    
    messages_length = len(state["messages"])
    print(f"Messages length: {messages_length}")
    
    #Append to file
    text = "User: " + state["messages"][-1].content + "\nAI: " + response.content
    destination = "elden_ring/characters/varre/testing/basic_rag_memory.txt"
    append_to_txt(destination, text)
    
    
    return {"messages": response}

def trim_messages(state: State):
    global vector_store
    
    copied_messages = state["messages"][:]
    
    current_total_tokens = count_tokens(copied_messages)
    print(f"Current token count: {current_total_tokens}")
    
    max_tokens = 100
    i = 0
    delete_messages = []
    
    while current_total_tokens > max_tokens and i < len(copied_messages) - 1:
        if isinstance(copied_messages[i], HumanMessage):
            while i < len(copied_messages) - 1 and isinstance(copied_messages[i], HumanMessage):
                i += 1
        if isinstance(copied_messages[i], AIMessage):
            while i < len(copied_messages) - 1 and isinstance(copied_messages[i], AIMessage):
                i += 1
        
        delete_messages = [RemoveMessage(id=m.id) for m in copied_messages[:i]]
        current_total_tokens = count_tokens(copied_messages[i:])
    
    
    # Add messages to RAG
    character = state["character"]
    long_term_memory = ""
    for m in copied_messages[:i]:
        if isinstance(m, HumanMessage):
            long_term_memory += "User: " + m.content + "\n"
        elif isinstance(m, AIMessage):
            long_term_memory += f"{character}: " + m.content + "\n"
        else:
            long_term_memory += f"Unkown: " + m.content + "\n"
    
    print("Messages that will be deleted and added to long term memory:")
    print(long_term_memory)
    vector_store.add_texts([long_term_memory])
    
    if i != 0:
        print(f"Exceeded max token count, Trimming...\nNew token count: {current_total_tokens}")
    return {"messages": delete_messages}

def retrieve(state: State):    
    
    rag_query = state["messages"][-1].content
    print(f"Rag Query: {rag_query}")
    documents = retriever.invoke(rag_query, k=10)
    
    for res in documents:
        print(f"{res.page_content}")
    
    return {"documents": documents}


In [12]:
workflow = StateGraph(State)

workflow.add_node("retriever", retrieve)
workflow.add_edge(START, "retriever")

workflow.add_node("trimmer", trim_messages)
workflow.add_edge("retriever", "trimmer")

workflow.add_node("model", call_model)
workflow.add_edge("trimmer", "model")

workflow.add_edge("model", END)

app = workflow.compile(checkpointer=memory)

Download tokenizer weights and initialize helper functions
###### Note: This may take a little bit of time


In [7]:
import cohere  

with open(f'api.txt', errors='ignore') as f:
    api_key = f.read()
co = cohere.ClientV2(api_key=api_key)

tokenized_output = co.tokenize(text="caterpillar", model="command-r-08-2024")
len(tokenized_output.tokens)

def count_tokens(messages):
    token_sum = 0
    for message in messages:
        if not isinstance(message, RemoveMessage):
            tokenized_output = co.tokenize(text=message.content, model="command-r-08-2024")
            token_sum += len(tokenized_output.tokens)
    
    return token_sum

In [8]:
def append_to_txt(file_name, text):
    with open(file_name, 'a') as file:
        file.write(text + '\n')

Talk to to the model

In [13]:
query = "My name is Bob"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Rag Query: My name is Bob


Number of requested results 10 is greater than number of elements in index 2, updating n_results = 2




Current token count: 12
Messages that will be deleted and added to long term memory:


Displaying message type order:
HumanMessage
HumanMessage
HumanMessage


Documents found!
Messages length: 3
================================== Ai Message ==================================

Oh, Bob, is it? Well, Bob, you've certainly got a name. A name that's yours and yours alone. Or so you'd like to think. Names are such a fickle thing, aren't they? One day you're a Bob, the next, you're a nameless corpse.


In [ ]:
query = "Hello!"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

In [ ]:
query = "What is my name?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

In [ ]:
query = "How many times have I said hello?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

In [ ]:
query = "How are you?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

In [ ]:
query = "Are you sure you do not know my name?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

In [14]:
graph_state = app.get_state(config)
graph_state

StateSnapshot(values={'messages': [HumanMessage(content='My name is Bob', additional_kwargs={}, response_metadata={}, id='86d752bc-928d-4e9b-938d-8ab25add0168'), HumanMessage(content='My name is Bob', additional_kwargs={}, response_metadata={}, id='9c072e71-74cc-42bf-8115-c47631f4a3f4'), HumanMessage(content='My name is Bob', additional_kwargs={}, response_metadata={}, id='2dcc488a-92f8-45d3-a0b1-72fcd8365de4'), AIMessage(content="Oh, Bob, is it? Well, Bob, you've certainly got a name. A name that's yours and yours alone. Or so you'd like to think. Names are such a fickle thing, aren't they? One day you're a Bob, the next, you're a nameless corpse.", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '7b95997a-0645-4cd6-bdda-beb722dee1b3', 'token_count': {'input_tokens': 3444.0, 'output_tokens': 64.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'sear

In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

display(
    Image(
        app.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)